In [6]:
import json
import pandas as pd
import os
from pandas import DataFrame, Series
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(hosts="http://localhost:9200")
es.info()

/tmp/ipykernel_11127/3737662446.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.info()


ObjectApiResponse({'name': '547806deed1a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'lzis4wTMRMS_Y6iIGOWrMA', 'version': {'number': '7.15.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '93d5a7f6192e8a1a12e154a2b81bf6fa7309da0c', 'build_date': '2021-11-04T14:04:42.515624022Z', 'build_snapshot': False, 'lucene_version': '8.9.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [39]:
# If the index has been created before it can be removed by this line
# es.indices.delete(index='komoot')

/tmp/ipykernel_11127/1674787250.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.indices.delete(index='komoot')


ObjectApiResponse({'acknowledged': True})

In [40]:
# make start_point and coordinates geo types useable in kibana
mapping = {
    "mappings": {
        "properties": {
            "start_point": {
                "type": "geo_point"
            },
            "point_track": {
                "type": "geo_shape"
            }
        }
    }
}

es.indices.create(
    index="komoot",
    body=mapping
)

/tmp/ipykernel_11127/3794689600.py:14: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(
/tmp/ipykernel_11127/3794689600.py:14: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  es.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'komoot'})

In [50]:
# load tour details from jsons
tours = []
for subdir, dirs, files in os.walk("data"):
    for file in files:
        filepath = subdir + os.sep + file

        if filepath.endswith(".json"):
            tour_id = filepath.replace(".json", "").split("/")[-1]
            with open(filepath) as f:
                tours.append(json.load(f))

In [42]:
def extract_coordinates(points):
    coordinates = [[x["lng"],x["lat"]] for x in points]
    return {"type": "LineString", "coordinates": coordinates}

In [51]:
df = DataFrame(tours)
df['start_point'] = df['start_point']\
    .map(lambda item: [item['lng'], item['lat']])
df["point_track"] = df.apply(lambda x: extract_coordinates(x["_embedded"]["coordinates"]["items"]), axis=1)
df = df.drop(["_embedded", "path"], axis=1)
df.head()

,status,type,date,name,source,distance,duration,sport,query,constitution,...,elevation_down,tour_information,id,changed_at,vector_map_image,vector_map_image_preview,map_image,map_image_preview,time_in_motion,point_track
0,private,tour_planned,2020-07-18T10:36:19.753+02:00,Unterer Wasserfallweg – Aussichtsturm Hohe War...,"{""api"":""de.komoot.smarttour"",""type"":""tour_plan...",25618.000000,28156,hike,d01AuSoEgCNwZc=FxjUEiwkiZ3ZvQQYF7s5WuKzhOqe_-Q...,3.0,...,774.000000,[],221418774,2020-07-18T08:36:21.057Z,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,NaN,"{'type': 'LineString', 'coordinates': [[9.2901..."
1,private,tour_recorded,2022-07-09T09:14:23.687+02:00,Von Alpirsbach nach Freiburg im Breisgau,"{""api"":""de.komoot.main-api/tour/recorded"",""typ...",50938.072056,16108,touringbicycle,NaN,NaN,...,580.899786,NaN,837562793,2022-07-10T07:43:27.867Z,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,11801.0,"{'type': 'LineString', 'coordinates': [[9.1542..."
2,private,tour_recorded,2020-10-30T10:48:10.877+01:00,Gohrischstein – Papststein Runde von Neue Haup...,"{""api"":""de.komoot.main-api/tour/recorded"",""typ...",83.428723,132,hike,NaN,NaN,...,0.000000,NaN,278205377,2020-10-30T09:50:26.774Z,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,103.0,"{'type': 'LineString', 'coordinates': [[14.116..."
3,private,tour_planned,2020-11-21T12:36:24.939+01:00,Blaustrümpflerweg mit Seilbahn und Zacke 22.03...,"{""api"":""de.komoot.main-api/tour/recorded"",""typ...",9855.000000,10511,hike,d01AufiYACLkdY=FvcAh2_zsAZ_rX9v_d_97-2f93-iAP_...,3.0,...,300.000000,"[{'type': 'OFF_GRID', 'segments': [{'from': 0,...",287297552,2020-11-21T11:36:25.701Z,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,NaN,"{'type': 'LineString', 'coordinates': [[9.1468..."
4,private,tour_recorded,2022-07-09T09:13:29.249+02:00,Fahrradtour,"{""api"":""de.komoot.main-api/tour/recorded"",""typ...",96432.740905,40043,touringbicycle,NaN,NaN,...,1302.100198,NaN,837117009,2022-07-09T18:21:05.008Z,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,{'src': 'https://tourpic-vector.maps.komoot.ne...,23746.0,"{'type': 'LineString', 'coordinates': [[9.1541..."


In [53]:
# index tour data
for i, row in df.iterrows():
    res = es.index(index='komoot', id=row['id'], document=row.to_json())

/tmp/ipykernel_11127/1338930309.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  res = es.index(index='komoot', id=row['id'], document=row.to_json())
